# DNSSEC-Aware Resolver Downgrade Attacks

This notebook contains data collection and analysis code to study resolver populations ("groups") for their vulnerability towards downgrade attacks.

Prerequisite for running this notebook is a properly setup test zone at `downgrade.dedyn.io` (see `ZONE` variable below).

In [1]:
import logging
import random
import string
from datetime import datetime
import itertools
import concurrent
import math

import numpy as np
import dns.message, dns.query, dns.rdataclass, dns.rdatatype, dns.flags, dns.exception, dns.name, dns.dnssec
from tqdm import tqdm
import pandas as pd
import requests

IN = dns.rdataclass.from_text("IN")
NS = dns.rdatatype.from_text("NS")
SOA = dns.rdatatype.from_text("SOA")
DS = dns.rdatatype.from_text("DS")
A = dns.rdatatype.from_text("A")
TXT = dns.rdatatype.from_text("TXT")
AAAA = dns.rdatatype.from_text("AAAA")
RRSIG = dns.rdatatype.from_text("RRSIG")

ALGORITHMS = [
    # all relevant DNSSEC algorithms
    dns.dnssec.RSASHA1,
    dns.dnssec.RSASHA256,
    dns.dnssec.RSASHA512,
    dns.dnssec.ECDSAP256SHA256,
    dns.dnssec.ECDSAP384SHA384,
    dns.dnssec.ED25519,
    dns.dnssec.ED448,
]

ALGORITHMS_RED = ALGORITHMS = [
    # all DNSSEC algorithms used in this study
    dns.dnssec.RSASHA1,
    dns.dnssec.RSASHA256,
    dns.dnssec.ECDSAP256SHA256,
    dns.dnssec.ED25519,
    dns.dnssec.ED448,
]

ZONE = dns.name.from_text('downgrade.dedyn.io')

executor = concurrent.futures.ThreadPoolExecutor(1)  # increase number of workers to decrease runtime of the study at the risk of overloading the Internet connection and/or auth NS

def query(qname, resolver, cd, rdtype=A):
    """Wrapper method to query resolvers for data. Retries once on timeouts."""
    q = dns.message.make_query(qname, rdtype)
    q.flags |= dns.flags.AD
    if cd:
        q.flags |= dns.flags.CD
    
    if resolver.startswith('https'):
        method = dns.query.https
        where = resolver
    elif resolver.startswith('tls'):
        method = dns.query.tls
        where = resolver[len('tls://'):]
    else:
        method = dns.query.udp
        where = resolver
        
    logging.info(f'Query:\n{q}')
    
    try:
        return method(q, where=where, timeout=2)
    except (dns.exception.Timeout, requests.exceptions.ReadTimeout, EOFError):
        return method(q, where=where, timeout=5)
    
def run(f, args_list):
    """Calls function f for each args in args_list once, using multi-threading."""
    results = []    
    try:
        futures = [executor.submit(f, *args) for args in args_list]
        with tqdm(total=len(futures)) as pbar:
            for future in concurrent.futures.as_completed(futures):
                pbar.update(1)
                if future.exception():
                    logging.warning(f"{future.exception()}")
                    results.append({'status': future.exception()})
                else:
                    results.append(future.result())
    finally:
        return results    

## Define Test Zones with Different Combinations of DS and DNSKEY Records

Our study runs on zones with various DS and DNSKEY configurations. Which exactly is determined below. Note that these zones must exist and may need to be configured at the auth NS.

In [3]:
zones = [
    {
        'ds': algos, 
        'dnskey': tuple(sorted(set(algos) - set(remove_dnskeys))),
        'name': dns.name.from_text(
            "-".join(
                [f"ds{a}" for a in sorted(algos)] +
                [f"dnskey{int(a)}" for a in sorted(set(algos) - set(remove_dnskeys))]
            ),
            origin=ZONE
        ),
    }
    for algos in itertools.chain(itertools.combinations(ALGORITHMS, 1), itertools.combinations(ALGORITHMS_RED, 2))
    for remove_dnskeys in [[a for i, a in enumerate(algos) if v[i]] for v in itertools.product([True, False], repeat=len(algos))]
    #if 16 in algos
]
zones = pd.DataFrame(zones)
zones = zones.set_index('name')
zones

,ds,dnskey
name,,
"(b'ds5', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA1,)",()
"(b'ds5-dnskey5', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA1,)","(Algorithm.RSASHA1,)"
"(b'ds8', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA256,)",()
"(b'ds8-dnskey8', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.RSASHA256,)","(Algorithm.RSASHA256,)"
"(b'ds13', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ECDSAP256SHA256,)",()
"(b'ds13-dnskey13', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ECDSAP256SHA256,)","(Algorithm.ECDSAP256SHA256,)"
"(b'ds15', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED25519,)",()
"(b'ds15-dnskey15', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED25519,)","(Algorithm.ED25519,)"
"(b'ds16', b'downgrade', b'dedyn', b'io', b'')","(Algorithm.ED448,)",()


## Define Resolver Populations to be Studied

Creates a list of resolvers to be studied and determines their algorithm support.

In [4]:
open_resolvers = [{'resolver_addr': row['IPv4'], 'resolver_name': row['Handle'], 'resolver_group': 'open-named'} for _, row in pd.read_csv("open-resolvers.csv").iterrows()]
lab_resolvers = [{'resolver_addr': row['IPv4'], 'resolver_name': row['Handle'], 'resolver_group': 'lab'} for _, row in pd.read_csv("lab-resolvers.csv").iterrows()]

In [ ]:
def convert_resolver_format(d):
    return [{'resolver_addr': addr, 'resolver_name': handle, 'resolver_group': 'open-named'} for handle, addr in d.items()]
    
doh_resolvers = convert_resolver_format({
    'cloudflare-doh': 'https://cloudflare-dns.com/dns-query',
    'cloudflare-mozilla-doh': 'https://mozilla.cloudflare-dns.com/dns-query',
    'google-doh': 'https://dns.google/dns-query',
    'quad9-doh': 'https://dns.quad9.net/dns-query',
    'clean-browsing-doh': 'https://security-filter-dns.cleanbrowsing.org/dns-query',
    'adguard-doh': 'https://dns.adguard.com/dns-query',
    'comcast-doh': 'https://doh.xfinity.com/dns-query',
})
dot_resolvers = convert_resolver_format({
    'cloudflare-dot': 'tls://1.1.1.1',
    'google-dot': 'tls://8.8.8.8',
    'quad9-dot': 'tls://9.9.9.9',
    'clean-browsing-dot': 'tls://185.228.168.9',
    'adguard-dot': 'tls://94.140.14.14',
})

In [6]:
def resolver_transport(row):
    if row['resolver_addr'].startswith('tls'):
        return 'DoT'
    if row['resolver_addr'].startswith('https'):
        return 'DoH'
    return 'UDP/TCP'

resolver_list = pd.DataFrame(
    open_resolvers + lab_resolvers + 
    doh_resolvers + dot_resolvers 
)
resolver_list['resolver_transport'] = resolver_list.apply(resolver_transport, axis=1)
resolver_list.head(5)

,resolver_addr,resolver_name,resolver_group,resolver_transport
0,208.67.222.222,cisco-umbrella,open-named,UDP/TCP
1,1.1.1.1,cloudflare,open-named,UDP/TCP
2,8.26.56.26,comodo-secure-dns,open-named,UDP/TCP
3,193.17.47.1,cznic-odvr,open-named,UDP/TCP
4,80.80.80.80,freenom-world,open-named,UDP/TCP


### Determine Resolver Cipher Support

In [7]:
def check_resolver(resolver, algorithm):
    try:
        qname = dns.name.from_text(f'mitm-ms.ds{algorithm}-dnskey{algorithm}', origin=ZONE)
        
        r = query(qname, resolver['resolver_addr'], cd=False, rdtype=TXT)  # signature invalid
        
        return {
            **resolver,
            'algorithm': algorithm,
            'status': 'ok',
            'qname': qname.to_text(),
            'ad': dns.flags.AD in r.flags,
            'rcode': r.rcode()
        }
    except (dns.exception.Timeout, requests.exceptions.ReadTimeout, EOFError):
        return {
            **resolver,
            'algorithm': algorithm,
            'status': 'timeout',
        }
    except Exception as e:
        return {
            **resolver,
            'algorithm': algorithm,
            'status': (type(e), e),
        }

In [8]:
logging.basicConfig(level=logging.WARNING, force=True)

In [9]:
resolver_support_results = run(check_resolver, [(resolver, a) for _, resolver in resolver_list.iterrows() for a in ALGORITHMS])
resolver_support = pd.DataFrame(resolver_support_results)

100%|██████████| 165/165 [01:10<00:00,  2.35it/s]


In [10]:
def support(row):
    def log():
        logging.warning(f'Weird resolver behavior for {row["resolver_name"]}: {row["qname1"]} -> {row["rcode1"]}, {row["qname2"]} -> {row["rcode2"]}')
        
    if row['status'] != 'ok':
        return None
    
    return row['rcode'] == dns.rcode.Rcode.SERVFAIL
    
resolver_support['supported'] = resolver_support.apply(support, axis=1)

In [11]:
def uncertain_any(s):
    if None in list(s):  # None in s is always false, likely due to pandas' messing with the 'in' operator
        return None
    else:
        return any(s)
    
grouped = resolver_support.groupby(['resolver_addr', 'algorithm'], dropna=False)[['supported']].agg({
    'supported': [uncertain_any]
}).reset_index().pivot(index='resolver_addr', columns='algorithm', values=('supported', 'uncertain_any')).reset_index()
grouped.columns = ['resolver_addr'] + [f'supports_{a}' for a in ALGORITHMS]
resolvers = grouped.set_index('resolver_addr').join(resolver_list.set_index('resolver_addr'))

In [12]:
resolvers.loc['9.9.9.9', 'supports_16'] = False
resolvers.loc['tls://9.9.9.9', 'supports_16'] = False
resolvers.loc['https://dns.quad9.net/dns-query', 'supports_16'] = False

resolvers['support'] = resolvers.apply(lambda row: tuple(a for a in ALGORITHMS if row[f'supports_{a}'] is True), axis=1)

In [13]:
def row_style(row):
    styles = {
        True: 'color: green;',
        False: 'color: red;',
    }
    return [styles.get(v) for v in row]
    
order = ['resolver_group', 'resolver_name', 'resolver_transport']
resolvers.reset_index().set_index(order).sort_values(order).style.apply(row_style, axis=1)

In [14]:
resolvers['validating'] = resolvers.apply(lambda row: bool(row['support']), axis=1)
resolvers = resolvers[resolvers['validating']]

In [15]:
RESOLVER_NAMES = {
    'bind9113': 'Bind v9.11.3',
    'kresd532': 'Knot Resolver 5.3.2',
    'powerdns460': 'Power DNS Recursor 4.6.0',
    'unbound167': 'Unbound 1.6.7',
    'ws2012': 'Windows Server 2012',
    'ws2012r2': 'Windows Server 2012 R2',
    'ws2016': 'Windows Server 2016',
    'ws2019': 'Windows Server 2019',
    'adguard': 'AdGuard Public DNS',
    'cloudflare': 'Cloudflare Resolver',
    'cloudflare-mozilla': 'Cloudflare Resolver for Mozilla',
    'comcast': 'Comcast Public DNS',
    'google': 'Google Public DNS',
    'quad9': 'Quad9 Resolver',
    'cisco-umbrella': 'Cisco Umbrella',
    'comodo-secure-dns': 'Comodo Secure DNS',
    'cznic-odvr': 'cznic ODVR',
    'freenom-world': 'Freenom World',
    'oracle-dyn': 'Oracle Dyn',
    'yandex': 'Yandex safe'
}

RESOLVER_GROUPS = {
    'lab': 'Lab',
    'open-named': 'Public DNS',
}
ALGORITHM_NAMES = [f"{dns.dnssec.Algorithm.to_text(a)} ({str(int(a))})" for a in ALGORITHMS]
ALGORITHM_NUMBERS = [f"{str(int(a))}" for a in ALGORITHMS]

In [16]:
def single_value(s):
    assert len(s) == 1
    return s[0]

def removesuffix(s, suf):
    if s[-len(suf):] == suf:
        return s[:-len(suf)]
    return s

by = ['resolver_group', 'resolver_name']  # resolver_addr
t = resolvers[resolvers['resolver_transport'] == 'UDP/TCP'].sort_values(by).groupby(by).agg({
    f'supports_{a}': [single_value]
    for a in ALGORITHMS
}).reset_index()
t.columns = ['Group', 'Resolver'] + ALGORITHM_NUMBERS
del t['Group']
formatters = {
    algorithm_name: lambda val: {True: r'\cmark', False: r'\xmark', None: '??'}[val]
    for algorithm_name in ALGORITHM_NUMBERS
}
formatters.update({
    'Group': lambda s: RESOLVER_GROUPS.get(s, s),
    'Resolver': lambda s: RESOLVER_NAMES.get(removesuffix(removesuffix(s, '-dot'), '-doh'), s),
})
print(t.to_latex(index=False, formatters=formatters, escape=False, column_format='lllcccc'))

\begin{tabular}{lllcccc}
\toprule
                Resolver &      5 &      8 &     13 &     15 &     16 \\
\midrule
            Bind v9.11.3 & \cmark & \cmark & \cmark & \xmark & \xmark \\
     Knot Resolver 5.3.2 & \cmark & \cmark & \cmark & \xmark & \xmark \\
Power DNS Recursor 4.6.0 & \cmark & \cmark & \cmark & \cmark & \cmark \\
           Unbound 1.6.7 & \cmark & \cmark & \cmark & \cmark & \xmark \\
     Windows Server 2012 & \cmark & \cmark & \cmark & \cmark & \cmark \\
  Windows Server 2012 R2 & \cmark & \cmark & \cmark & \xmark & \xmark \\
     Windows Server 2016 & \cmark & \cmark & \cmark & \xmark & \xmark \\
     Windows Server 2019 & \cmark & \cmark & \cmark & \xmark & \xmark \\
          Cisco Umbrella & \cmark & \cmark & \cmark & \cmark & \cmark \\
     Cloudflare Resolver & \cmark & \cmark & \cmark & \cmark & \xmark \\
       Comodo Secure DNS & \cmark & \cmark & \cmark & \cmark & \cmark \\
              cznic ODVR & \cmark & \cmark & \cmark & \cmark & \xmark \\
        

## Define Attack Strategies

For each zone and each resolver, we run a number of different attack strategies. Which exactly is determined below.

In [17]:
attacks = [
    {'name': 'replace signature number with 253 (PRIVATEDNS) and fake content', 'instructions': ('rs17', 'at')},
    {'name': 'replace signature number with 17 (unassigned) and fake content', 'instructions': ('rs253', 'at')},
    {'name': 'replace signature number with ed448 and fake content', 'instructions': ('rs16', 'at')},
    {'name': 'replace signature number with ed25519 and fake content', 'instructions': ('rs15', 'at')},
    {'name': 'replace signature number with ecdsap256sha256 and fake content', 'instructions': ('rs13', 'at')},
    {'name': 'replace signature number with rsasha256 and fake content', 'instructions': ('rs8', 'at')},
    {'name': 'remove all signatures except ed448 and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS if a < dns.dnssec.ED448)},
    {'name': 'remove all signatures except ed25519 and ed448 and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS if a < dns.dnssec.ED25519)},
    {'name': 'strip all signatures and fake content', 'instructions': ('at',) + tuple(f'ds{a}' for a in ALGORITHMS)},
    {'name': 'invalidate signature', 'instructions': ('ms',),}
]
attacks = pd.DataFrame(attacks)
attacks['prefix'] = attacks.apply(lambda row: f"mitm-{'-'.join(row['instructions'])}", axis=1)
attacks = attacks.set_index('prefix')
attacks

,name,instructions
prefix,,
mitm-rs17-at,replace signature number with 253 (PRIVATEDNS)...,"(rs17, at)"
mitm-rs253-at,replace signature number with 17 (unassigned) ...,"(rs253, at)"
mitm-rs16-at,replace signature number with ed448 and fake c...,"(rs16, at)"
mitm-rs15-at,replace signature number with ed25519 and fake...,"(rs15, at)"
mitm-rs13-at,replace signature number with ecdsap256sha256 ...,"(rs13, at)"
mitm-rs8-at,replace signature number with rsasha256 and fa...,"(rs8, at)"
mitm-at-ds5-ds8-ds13-ds15,remove all signatures except ed448 and fake co...,"(at, ds5, ds8, ds13, ds15)"
mitm-at-ds5-ds8-ds13,remove all signatures except ed25519 and ed448...,"(at, ds5, ds8, ds13)"
mitm-at-ds5-ds8-ds13-ds15-ds16,strip all signatures and fake content,"(at, ds5, ds8, ds13, ds15, ds16)"


## Run Attack Evaluation

Here, we define how to collect data for a given resolver, zone config, and attack; then we run the check for all combinations of resolvers, zones, and attacks.

In [18]:
def check_attack(addr, prefix, zone):
    try:
        qname = dns.name.from_text(prefix, origin=zone)
        r1 = query(qname, addr, cd=False, rdtype=TXT)
        logging.info(f'Response:\n{r1}')
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': 'ok',
            'rcode': r1.rcode(),
            'response': r1,
            'evil_content': 'evil' in str(r1) or 'ms-' in qname.to_text(),
        }
    except dns.exception.Timeout:
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': 'timeout',
        }
    except Exception as e:
        logging.warning(f"Exception: {type(e).__name__}: {e}")
        return {
            'resolver_addr': addr,
            'zone': zone,
            'attack': prefix,
            'status': e,
        }

The following cell runs the attack data collection for the product of `attacks`, `zones`, and `resolvers`. Depending on the size of these lists, and on the multi-threading configuration, this may take minutes to hours.

In [19]:
attack_results = run(check_attack, [(addr, prefix, zone) for prefix, _ in attacks.iterrows() for zone, _ in zones.iterrows() for addr, _ in resolvers.iterrows()])
attack_results = pd.DataFrame(attack_results)

100%|██████████| 13500/13500 [51:16<00:00,  4.39it/s]


In [35]:
# combine attack data with details on resolvers, zones, and attacks
results = attack_results.join(resolvers, on='resolver_addr').join(zones, on='zone').join(attacks, on='attack')
assert len(attack_results) == len(results), (len(attack_results), len(results)) # make sure this worked as expected

In [36]:
# save raw data
results.to_pickle(datetime.now().strftime("results-%Y-%m-%d--%H-%M-%S.pickle"))

In [37]:
# load data
#results = pd.read_pickle("results-2021-10-11--15-25-17.pickle")

In [43]:
# extract and enrich data
results['status_str'] = results.apply(lambda row: str(row['status']), axis=1)
results['supported_ds'] = results.apply(lambda row: tuple(set(row['ds']) & set(row['support'])), axis=1)
results['supported_dnskey'] = results.apply(lambda row: tuple(set(row['dnskey']) & set(row['support'])), axis=1)
results['validation_paths'] = results.apply(lambda row: tuple(set(row['dnskey']) & set(row['ds'])), axis=1)
results['qname'] = results.apply(lambda row: dns.name.from_text(row['attack'], origin=row['zone']), axis=1)
results['evil_content'] = results['qname'].apply(lambda x: '-ms' in x.to_text()) | results['evil_content']
results['zone_prefix'] = results.apply(lambda row: row['zone'][0].decode(), axis=1)
results['zone_config'] = results.apply(lambda row: f"DS: {','.join(str(int(e)) for e in row['ds'])} DNSKEY: {','.join(str(int(e)) for e in row['dnskey'])}", axis=1)
results['zone_name'] = results.apply(lambda row: row['zone'].to_text(), axis=1)

def rrsig(row):
    ALGORITHMS = range(255)
    if '-rs' in row['attack'] and '-ds' in row['attack']:
        raise NotImplemented
    if '-rs' in row['attack']:
        for a in reversed(ALGORITHMS):  # reverse to avoid matching rs1 instead of rs10
            if f'-rs{a}' in row['attack']:
                return tuple([a])
    if '-ds' in row['attack']:
        return tuple(set(row['ds']) - {a for a in ALGORITHMS if f'ds{a}' in row['attack']})
    return row['ds']

results['rrsig'] = results.apply(rrsig, axis=1)

results['num_ds'] = results.apply(lambda row: len(row['ds']), axis=1)
results['num_supported_ds'] = results.apply(lambda row: len(row['supported_ds']), axis=1)
results['num_unsupported_ds'] = results.apply(lambda row: row['num_ds'] - row['num_supported_ds'], axis=1)
results['num_dnskey'] = results.apply(lambda row: len(row['dnskey']), axis=1)
results['num_supported_dnskey'] = results.apply(lambda row: len(row['supported_dnskey']), axis=1)
results['num_unsupported_dnskey'] = results.apply(lambda row: row['num_dnskey'] - row['num_supported_dnskey'], axis=1)
results['supported_rrsig'] = results.apply(lambda row: tuple(set(row['rrsig']) & set(row['support'])), axis=1)
results['num_rrsig'] = results.apply(lambda row: len(row['rrsig']), axis=1)
results['num_supported_rrsig'] = results.apply(lambda row: len(row['supported_rrsig']), axis=1)
results['num_unsupported_rrsig'] = results.apply(lambda row: row['num_rrsig'] - row['num_supported_rrsig'], axis=1)
results['has_supported'] = results.apply(lambda row: row['num_rrsig'] - row['num_supported_rrsig'], axis=1)
results['has_supported_ds'] = results.apply(lambda row: bool(row['num_supported_ds']), axis=1)
results['has_unsupported_ds'] = results.apply(lambda row: bool(row['num_unsupported_ds']), axis=1)
results['has_supported_rrsig'] = results.apply(lambda row: bool(row['num_supported_rrsig']), axis=1)
results['has_unsupported_rrsig'] = results.apply(lambda row: bool(row['num_unsupported_rrsig']), axis=1)

def ds_support_status(row):
    if row['has_supported_ds'] and row['has_unsupported_ds']:
        return 'both'
    if row['has_supported_ds']:
        return 'supported'
    if row['has_unsupported_ds']:
        return 'unsupported'
    return 'none'
    
results['ds_support_status'] = results.apply(ds_support_status, axis=1)

def rrsig_support_status(row):
    if row['has_supported_rrsig'] and row['has_unsupported_rrsig']:
        return 'both'
    if row['has_supported_rrsig']:
        return 'supported'
    if row['has_unsupported_rrsig']:
        return 'unsupported'
    return 'none'
    
results['rrsig_support_status'] = results.apply(rrsig_support_status, axis=1)

def rrsig_dangling_status(row):
    if set(row['rrsig']).issubset(row['ds']):
        return 'no dangling'
    if set(row['ds']).issubset(row['rrsig']):
        return 'some sangling'
    return 'all dangling'
    
results['rrsig_dangling_status'] = results.apply(rrsig_dangling_status, axis=1)

def poc(row):
    if row['behavior_correct']:
        return None
    resolver_addr = row['resolver_addr']
    attack = row['attack']
    zone_prefix = row['zone_prefix']
    resolver_transport = row['resolver_transport']
    if resolver_transport == 'UDP/TCP':
        return f"dig @{resolver_addr} TXT +dnssec {attack}.{zone_prefix}.downgrade.dedyn.io"
    elif resolver_transport == 'DoH':
        resolver_addr = resolver_addr.split('//', 1)[1].split('/', 1)[0]
        return f"kdig @{resolver_addr} TXT +dnssec +https {attack}.{zone_prefix}.downgrade.dedyn.io"
    elif resolver_transport == 'DoT':
        return f"kdig TXT @{resolver_addr} +tls-ca +tls-host={resolver_addr} {attack}.{zone_prefix}.downgrade.dedyn.io"
    logging.warning(f'No POC for {resolver_transport=}')
    
results['poc'] = results.apply(poc, axis=1)

results['Resolver'] = results.apply(lambda row: f"{RESOLVER_NAMES.get(removesuffix(removesuffix(row['resolver_name'], '-dot'), '-doh'), row['resolver_name'])} ({row['resolver_transport']})", axis=1)

results['ds_numbers'] = results.apply(lambda row: tuple(sorted(int(a) for a in row['ds'])), axis=1)
results['dnskey_numbers'] = results.apply(lambda row: tuple(sorted(int(a) for a in row['dnskey'])), axis=1)


To determine if the resolver behaved correctly for any given query, we look at the response code and if there was evil content in the response:

In [39]:
def behavior_correct(row):
    if row['status'] != 'ok':
        return None
    
    if not row['supported_ds']:
        # resolvers will treat zone as insecure if there is no supported DS algorithm
        # we don't care if resolvers give SERVFAIL more often than appropriate
        return True
    
    if row['rcode'] == dns.rcode.Rcode.NOERROR and row['evil_content']:
        # evil content present, i.e. the signature invalid, but response wasn't SERVFAIL
        return False
    elif row['rcode'] == dns.rcode.Rcode.SERVFAIL: # we don't see the content so cannot check for evil content:
        # desired behavior for invalid signatures
        # we don't care if resolvers give SERVFAIL more often than appropriate
        return True
    
    # something else we didnt expect?
    logging.warning(f"Don't know if behavior is correct for rcode={row['rcode']} evil_content={row['evil_content']} "
                    f"ds={', '.join(str(int(a)) for a in row['ds'])} "
                    f"supported_ds={', '.join(str(int(a)) for a in row['supported_ds'])} qname={row['qname']}")
    logging.warning(row['response'])
    
    return None

results['behavior_correct'] = results.apply(behavior_correct, axis=1)

## Successful Attacks

In [41]:
pd.options.display.max_rows = len(resolvers) * len(attacks)

def values(s):
    return '; '.join(s)

def zone_proportion(s):
    return len(s) / len(zones)

attack_success_rate = results.groupby(['attack', 'name', 'resolver_name', 'resolver_addr'], dropna=False).agg({
    'behavior_correct': [len, 'mean']
}).reset_index()
attack_success_rate[attack_success_rate[('behavior_correct', 'mean')] < 1].head(10)

attack                                               name  \
                                                                              
0   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
1   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
5   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
6   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
7   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
8   mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
13  mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
14  mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
15  mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   
23  mitm-at-ds5-ds8-ds13  remove all signatures except ed25519 and ed448...   

             resolver_name                                 resolver_addr  \
                                                                           
0              adguard-doh             https://dns.adguard.com/dns-query   
1              adguard-dot                            tls://94.140.14.14   
5               cloudflare                                       1.1.1.1   
6           cloudflare-doh          https://cloudflare-dns.com/dns-query   
7           cloudflare-dot                                 tls://1.1.1.1   
8   cloudflare-mozilla-doh  https://mozilla.cloudflare-dns.com/dns-query   
13                  google                                       8.8.8.8   
14              google-doh                  https://dns.google/dns-query   
15              google-dot                                 tls://8.8.8.8   
23                  ws2012                                 141.12.174.44   

   behavior_correct        
                len  mean  
0                50  0.82  
1                50  0.82  
5                50  0.84  
6                50  0.84  
7                50  0.84  
8                50  0.84  
13               50  0.82  
14               50  0.82  
15               50  0.82  
23               50  0.78

## Resolver Behavior Correctness (Over Attacks and Configurations)

In [42]:
pd.options.display.max_rows = len(resolvers) * len(attacks) * 2

def status_ok(s):
    return (s == 'ok').mean()

results.groupby(['resolver_group', 'resolver_name'], dropna=False).agg({
    'status': [status_ok],
    'behavior_correct': ['mean'],
})

status behavior_correct
                                      status_ok             mean
resolver_group resolver_name                                    
lab            bind9113                   1.000         1.000000
               kresd532                   1.000         1.000000
               powerdns460                1.000         1.000000
               unbound167                 1.000         1.000000
               ws2012                     1.000         0.786000
               ws2012r2                   0.996         0.843373
               ws2016                     1.000         0.844000
               ws2019                     1.000         0.844000
open-named     adguard-doh                1.000         0.660000
               adguard-dot                1.000         0.660000
               cisco-umbrella             0.970         1.000000
               clean-browsing-dot         1.000         1.000000
               cloudflare                 1.000         0.888000
               cloudflare-doh             1.000         0.888000
               cloudflare-dot             1.000         0.888000
               cloudflare-mozilla-doh     1.000         0.888000
               comcast-doh                1.000         1.000000
               comodo-secure-dns          0.996         1.000000
               cznic-odvr                 1.000         1.000000
               freenom-world              0.966         1.000000
               google                     1.000         0.660000
               google-doh                 1.000         0.660000
               google-dot                 1.000         0.660000
               oracle-dyn                 0.960         1.000000
               quad9                      1.000         1.000000
               quad9-doh                  1.000         1.000000
               quad9-dot                  0.986         1.000000

## Determine Conditions under which Resolvers are Vulnerable

In [58]:
def resolver_vuln(s):
    assert len(s) <= 1
    return bool(next(iter(s)))

vuln = 'resolver vulnerable under the following condition'
by = ['Resolver', 'ds_support_status', 'rrsig_dangling_status', 'rrsig_support_status']
affected_resolvers = results.sort_values(by[0]).groupby(by).agg({
    'behavior_correct': ['mean']
}).reset_index()
affected_resolvers.columns = affected_resolvers.columns.droplevel(1)
affected_resolvers = affected_resolvers.pivot(index=by[0], columns=by[1:], values=['behavior_correct'])
affected_resolvers.style.apply(lambda row: ['color: red' if v < 1 else 'color: grey' for v in row], axis=1)

In [59]:
def resolver_vuln(s):
    assert len(s) <= 1
    return bool(next(iter(s)))

vuln = 'resolver vulnerable under the following condition'
results['DS Alg. Supported'] = results['ds_support_status']
results['RRSIG Alg. Supported'] = results['rrsig_support_status']
by = ['Resolver', 'ds_support_status', 'rrsig_dangling_status', 'rrsig_support_status']
affected_resolvers = results[results['behavior_correct'] == False].sort_values(by[0]).groupby(by).agg({
    'behavior_correct': ['min']
}).reset_index()
affected_resolvers[vuln] = ~affected_resolvers[('behavior_correct', 'min')]
affected_resolvers.columns = affected_resolvers.columns.droplevel(1)
affected_resolvers = affected_resolvers.groupby(by).agg({
    vuln: [resolver_vuln]
}).reset_index()
affected_resolvers.columns = affected_resolvers.columns.droplevel(1)
affected_resolvers = affected_resolvers.pivot(index=by[0], columns=by[1:], values=[vuln])
affected_resolvers.style.apply(lambda row: [{True: 'color: red;', False: 'color: darkgreen;'}.get(v, 'color: grey;') for v in row], axis=1)

### Behavior for Given Resolver with Respect to Attack and DS/DNSKEY Configuration

In [60]:
given_resolver = '8.8.8.8'

The following table shows on the y-axis the DS and DNSKEY configuration of a zone and the attack on the x-axis. It is colored by the behavioral correctness of the resolver.

In [62]:
results[results['resolver_addr'] == given_resolver].sort_values(['ds_numbers', 'dnskey_numbers', 'attack']).groupby(['ds_numbers', 'dnskey_numbers', 'attack']).agg({
    'behavior_correct': ['mean', len]
}).reset_index().pivot(columns=[('attack', '')], index=[('ds_numbers', ''), ('dnskey_numbers', '')], values=[('behavior_correct', 'mean')]).style.apply(lambda row: ['background-color: red;' if val < 1 else None for val in row], axis=1)

The following table shows on the y-axis the DS, RRSIG support status with respect to `given_resolver` (after attack) and the attack on the x-axis. It is colored by the behavioral correctness of the resolver. **It only shows configurations with DS algos == DNSKEY algos.**

In [65]:
results['has_covered_rrsig'] = results.apply(lambda row: bool(set(row['ds']) & set(row['dnskey']) & set(row['rrsig'])), axis=1)

results[(results['resolver_addr'] == given_resolver) & (results['ds'] == results['dnskey'])].sort_values(
    ['num_supported_ds', 'num_unsupported_ds', 'num_supported_rrsig', 'num_unsupported_rrsig', 'has_covered_rrsig', 'attack']).groupby(
    ['has_supported_ds', 'has_unsupported_rrsig', 'has_covered_rrsig', 'has_supported_rrsig', 'attack']).agg({
    'behavior_correct': ['mean', len],
    'zone_config': [values],
}).reset_index().pivot(columns=[('attack', '')], index=[
    ('has_supported_ds', ''), ('has_unsupported_rrsig', ''), ('has_covered_rrsig', ''), ('has_supported_rrsig', '')], 
                       values=[('behavior_correct', 'mean')]).style.apply(lambda row: ['background-color: red;' if val < 1 else None for val in row], axis=1)

The following table shows on the y-axis the DS, RRSIG support status with respect to `given_resolver` (after attack) and the attack on the x-axis. It is colored by the behavioral correctness of the resolver. **It only shows configurations with DS algos != DNSKEY algos.**

In [66]:
results['has_covered_rrsig'] = results.apply(lambda row: bool(set(row['ds']) & set(row['dnskey']) & set(row['rrsig'])), axis=1)

results[(results['resolver_addr'] == given_resolver) & (results['ds'] != results['dnskey'])].sort_values(
    ['num_supported_ds', 'num_unsupported_ds', 'num_supported_rrsig', 'num_unsupported_rrsig', 'has_covered_rrsig', 'attack']).groupby(
    ['has_supported_ds', 'has_unsupported_rrsig', 'has_covered_rrsig', 'has_supported_rrsig', 'attack']).agg({
    'behavior_correct': ['mean', len],
    'zone_config': [values],
}).reset_index().pivot(columns=[('attack', '')], index=[
    ('has_supported_ds', ''), ('has_unsupported_rrsig', ''), ('has_covered_rrsig', ''), ('has_supported_rrsig', '')], 
                       values=[('behavior_correct', 'mean')]).style.apply(lambda row: ['background-color: red;' if val < 1 else None for val in row], axis=1)

## Vulnerable DS Configurations per Resolver

The following analysis shows the vulnerability of the resolvers with respect to any attack, conditioned on the DS configuration of the zone. It includes data of the prevalence of the DS configurations in the wild.

In [68]:
# TODO replace with Elias' data
# values taken from Crawler Tranco
tranco_ds_distribution = {(1,): 4,
 (3,): 1,
 (5,): 882,
 (5, 7): 2,
 (5, 7, 8): 1,
 (5, 8): 20,
 (5, 10): 2,
 (5, 12): 1,
 (5, 13): 7,
 (7,): 1472,
 (7, 8): 8,
 (7, 8, 13, 14): 1,
 (7, 10): 1,
 (7, 13): 9,
 (8,): 21963,
 (8, 10): 5,
 (8, 13): 23,
 (8, 14): 1,
 (10,): 710,
 (10, 13): 2,
 (10, 14): 1,
 (12,): 2,
 (13,): 17862,
 (13, 15): 1,
 (14,): 267,
 (15,): 2}
tranco_ds_total = sum(c for c in tranco_ds_distribution.values())

# values taken from Crawler TLD
tld_ds_distribution = {(5,): 29, (7,): 34, (7, 8): 4, (8,): 1225, (10,): 33, (13,): 45}
tld_ds_total = sum(c for c in tld_ds_distribution.values())

In [69]:
def row_style(row):
    return ['color: red;' if behavior_correct is True else 'color: grey;' for behavior_correct in row]

def vulnerable(row):
    return {
        True: False,
        False: True,
    }.get(row['behavior_correct'], None)

results['Group'] = results.apply(lambda row: RESOLVER_GROUPS.get(row['resolver_group'], row['resolver_group']), axis=1)
#results['Group'] = results['resolver_group']
results['Resolver'] = results.apply(lambda row: RESOLVER_NAMES.get(removesuffix(removesuffix(row['resolver_name'], '-dot'), '-doh'), row['resolver_name']), axis=1)
results['Transport'] = results['resolver_transport']
results['DS Algorithms'] = results['ds_numbers']
results['Vulnerable'] = results.apply(vulnerable, axis=1)

vulnerable = results[~results['Vulnerable'].isna()].groupby(['Group', 'Resolver', 'Transport', 'DS Algorithms']).agg({
    'Vulnerable': [any]
}).reset_index()
vulnerable.columns = vulnerable.columns.droplevel(1)
vulnerable = vulnerable.pivot(columns=['DS Algorithms'], index=['Group', 'Resolver', 'Transport'], values=['Vulnerable'])
#vulnerable.columns = [('',) + x if not isinstance(x[1], tuple) else x + ('',) for x in vulnerable.columns]
vulnerable.columns = pd.MultiIndex.from_tuples(
    [(x[0], ', '.join(str(a) for a in x[1])) + (f"{tranco_ds_distribution.get(x[1], 0)/tranco_ds_total*100:.0f}\%", f"{tld_ds_distribution.get(x[1], 0)/tld_ds_total*100:.0f}\%") for x in vulnerable.columns],
    names=vulnerable.columns.names + ['Prevalence in Tranco 1M', 'Prevalence in TLDs']
)
vulnerable = vulnerable.reset_index(['Group'])
del vulnerable['Group']
vulnerable.style.apply(row_style, axis=1)

Visual inspection of above table shows that the vulnerabilities do not depend on the transport, hence it is removed below.

In [70]:
vulnerable = results[results['Vulnerable'] == True].groupby(['Group', 'Resolver', 'DS Algorithms']).agg({
    'Vulnerable': [any]
}).reset_index()
vulnerable.columns = vulnerable.columns.droplevel(1)
vulnerable = vulnerable.pivot(columns=['DS Algorithms'], index=['Group', 'Resolver'], values=['Vulnerable'])
#vulnerable.columns = [('',) + x if not isinstance(x[1], tuple) else x + ('',) for x in vulnerable.columns]
vulnerable.columns = pd.MultiIndex.from_tuples(
    [(x[0], ', '.join(str(a) for a in x[1])) + (f"{tranco_ds_distribution.get(x[1], 0)/tranco_ds_total*100:.0f}\%", f"{tld_ds_distribution.get(x[1], 0)/tld_ds_total*100:.0f}\%") for x in vulnerable.columns],
    names=vulnerable.columns.names + ['Prevalence in Tranco 1M', 'Prevalence in TLDs']
)
vulnerable = vulnerable.reset_index(['Group'])
del vulnerable['Group']
vulnerable.style.apply(row_style, axis=1)

In [71]:
# print for paper
formatters = {
    k: lambda val: {True: r'\cmark', False: r'', None: '??'}[val]
    for k in vulnerable.keys()
}
print(vulnerable.to_latex(index=True, formatters=formatters, escape=False, na_rep=''))

\begin{tabular}{llllllllllllllll}
\toprule
{} & \multicolumn{15}{l}{Vulnerable} \\
DS Algorithms &          5 &    5, 8 &   5, 13 &   5, 15 &   5, 16 &       8 &   8, 13 &   8, 15 &   8, 16 &      13 &  13, 15 &  13, 16 &      15 &  15, 16 &      16 \\
Prevalence in Tranco 1M &        2\% &     0\% &     0\% &     0\% &     0\% &    51\% &     0\% &     0\% &     0\% &    41\% &     0\% &     0\% &     0\% &     0\% &     0\% \\
Prevalence in TLDs &        2\% &     0\% &     0\% &     0\% &     0\% &    89\% &     0\% &     0\% &     0\% &     3\% &     0\% &     0\% &     0\% &     0\% &     0\% \\
Resolver                        &            &         &         &         &         &         &         &         &         &         &         &         &         &         &         \\
\midrule
Windows Server 2012             &            &         &         &  \cmark &  \cmark &         &         &  \cmark &  \cmark &         &  \cmark &  \cmark &  \cmark &  \cmark &  \cmark \\
Windows

## Proof of Concepts for Vulnerabilities shown Above

In [72]:
def first_value(s):
    return next(iter(s))

results['ds_dnskey_match'] = results['ds'] == results['dnskey']
vulnerable = results[results['behavior_correct'] == False].sort_values(['ds_dnskey_match']).groupby(['resolver_group', 'resolver_name', 'ds_numbers']).agg({
    'poc': [first_value],
    'response': [first_value],
}).reset_index()
vulnerable.columns = vulnerable.columns.droplevel(1)
vulnerable = vulnerable.pivot(columns=['ds_numbers'], index=['resolver_group', 'resolver_name'], values=['poc', 'response'])
vulnerable

poc  \
ds_numbers                                                                                                                                      (5,)   
resolver_group resolver_name                                                                                                                           
lab            ws2012                                                                                                                            NaN   
               ws2012r2                                                                                                                          NaN   
               ws2016                                                                                                                            NaN   
               ws2019                                                                                                                            NaN   
open-named     adguard-doh                                      kdig @dns.adguard.com TXT +dnssec +https mitm-rs16-at.ds5-dnskey5.downgrade.dedyn.io   
               adguard-dot             kdig TXT @tls://94.140.14.14 +tls-ca +tls-host=tls://94.140.14.14 mitm-rs16-at.ds5-dnskey5.downgrade.dedyn.io   
               cloudflare                                                                                                                        NaN   
               cloudflare-doh                                                                                                                    NaN   
               cloudflare-dot                                                                                                                    NaN   
               cloudflare-mozilla-doh                                                                                                            NaN   
               google                                                           dig @8.8.8.8 TXT +dnssec mitm-rs17-at.ds5-dnskey5.downgrade.dedyn.io   
               google-doh                                            kdig @dns.google TXT +dnssec +https mitm-rs16-at.ds5-dnskey5.downgrade.dedyn.io   
               google-dot                        kdig TXT @tls://8.8.8.8 +tls-ca +tls-host=tls://8.8.8.8 mitm-rs16-at.ds5-dnskey5.downgrade.dedyn.io   

                                                                                                                                                          \
ds_numbers                                                                                                                                        (5, 8)   
resolver_group resolver_name                                                                                                                               
lab            ws2012                                                                                                                                NaN   
               ws2012r2                                                    dig @141.12.174.11 TXT +dnssec mitm-rs8-at.ds5-ds8-dnskey8.downgrade.dedyn.io   
               ws2016                                                      dig @141.12.174.42 TXT +dnssec mitm-rs8-at.ds5-ds8-dnskey8.downgrade.dedyn.io   
               ws2019                                                      dig @141.12.174.63 TXT +dnssec mitm-rs8-at.ds5-ds8-dnskey8.downgrade.dedyn.io   
open-named     adguard-doh                                      kdig @dns.adguard.com TXT +dnssec +https mitm-rs15-at.ds5-ds8-dnskey8.downgrade.dedyn.io   
               adguard-dot             kdig TXT @tls://94.140.14.14 +tls-ca +tls-host=tls://94.140.14.14 mitm-rs15-at.ds5-ds8-dnskey5.downgrade.dedyn.io   
               cloudflare                                                                                                                            NaN   
               cloudflare-doh                                                                                                                        NaN